# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import datetime
import pandas as pd
import numpy as np
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data = song_data.head(1).values[0]
song_data = song_data.tolist()
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
#  song_id, title, artist_id, year, duration
cur.execute(song_table_insert, tuple(song_data))
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
# {
#     "num_songs": 1,
#     "artist_id": "ARL7K851187B99ACD2",
#     "artist_latitude": null,
#     "artist_longitude": null,
#     "artist_location": "",
#     "artist_name": "Andy Andy",
#     "song_id": "SOMUYGI12AB0188633",
#     "title": "La Culpa",
#     "duration": 226.35057,
#     "year": 0
# }

In [10]:
# INSERT INTO artists (artist_id, name, location, latitude, longitude) VALUES (%s, %s, %s, %s, %s)
df = df.replace({np.nan: None})
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data = artist_data.head(1).values[0]
artist_data = artist_data.tolist()
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("data/log_data")

In [13]:
filepath = log_files[0]

In [14]:
df = pd.read_json(filepath, lines=True)
df = df.head(n=5)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
log_data = df.replace({np.nan: None})
log_data = log_data.loc[log_data['page'] == "NextSong"]

In [16]:
def millis_to_datetime(millis):
    return datetime.datetime.fromtimestamp(millis / 1000)

log_data['ts'] = df['ts'].map(millis_to_datetime)

In [17]:
def datetime_to_parts(d):
    return [
        d.timestamp(), d.hour, d.day, int(d.strftime("%W")), d.month, d.year, d.weekday()
    ]
    
log_data['time_data'] = log_data['ts'].map(datetime_to_parts)


In [18]:
log_data['column_labels'] = log_data['time_data'].map(lambda x: ['timestamp', 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday'])

In [19]:
time_df = log_data[['time_data', 'column_labels']]
# time_df.to_dict()
time_df

,time_data,column_labels
0,"[1541907236.796, 3, 11, 45, 11, 2018, 6]","[timestamp, hour, day, week_of_year, month, ye..."
1,"[1541907370.796, 3, 11, 45, 11, 2018, 6]","[timestamp, hour, day, week_of_year, month, ye..."
2,"[1541907634.796, 3, 11, 45, 11, 2018, 6]","[timestamp, hour, day, week_of_year, month, ye..."
4,"[1541914573.796, 5, 11, 45, 11, 2018, 6]","[timestamp, hour, day, week_of_year, month, ye..."


In [20]:
def merge(row): 
    time = row[0]
    labels = row[1]
    return dict(zip(labels, time))

time_df = time_df.copy()
dictionaries = time_df[['time_data', 'column_labels']].apply(merge, axis=1)
dictionaries = dictionaries.to_list()

In [21]:
time_df = pd.DataFrame.from_dict(dictionaries)

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [22]:
for i, row in time_df.iterrows():
    print(list(row))
    cur.execute(time_table_insert, list(row))
    conn.commit()

[1541907236.796, 3.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
[1541907370.796, 3.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
[1541907634.796, 3.0, 11.0, 45.0, 11.0, 2018.0, 6.0]
[1541914573.796, 5.0, 11.0, 45.0, 11.0, 2018.0, 6.0]


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [23]:
user_df = df.copy()
user_df = user_df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
3,32,Lily,Burns,F,free
4,32,Lily,Burns,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [24]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.